# Machine Learning Data Lineage with MLflow and Delta Lake

Dependencies:
* Please install `koalas` and `yellowbrick` Python libraries

In [0]:
import pandas as pd
import pyspark
import mlflow
import mlflow.sklearn

import warnings
warnings.filterwarnings("ignore")

## Data Source
We will start by reading in the [Kaggle Boston Housing dataset](https://www.kaggle.com/c/boston-housing/data).

![](https://storage.googleapis.com/kaggle-competitions/kaggle/5315/logos/front_page.png)

**Housing Values in Suburbs of Boston**
The medv variable is the target variable.

**Data description**
The Boston data frame has 506 rows and 14 columns.

This data frame contains the following columns:

| columns | description |
| :------- | :----------- |
| crim |  per capita crime rate by town. |
| zn | proportion of residential land zoned for lots over 25,000 sq.ft. |
| indus | proportion of non-retail business acres per town. |
| chas |  Charles River dummy variable (= 1 if tract bounds river; 0 otherwise). |
| nox |  nitrogen oxides concentration (parts per 10 million). |
| rm | average number of rooms per dwelling. |
| age |  proportion of owner-occupied units built prior to 1940. |
| dis |  weighted mean of distances to five Boston employment centres. |
| rad |  index of accessibility to radial highways. |
| tax | full-value property-tax rate per \$10,000. |
| ptratio | pupil-teacher ratio by town. |
| black | 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town. |
| lstat | lower status of the population (percent). |
| medv | median value of owner-occupied homes in \$1000s. |

Sources:
* Harrison, D. and Rubinfeld, D.L. (1978) Hedonic prices and the demand for clean air. J. Environ. Economics and Management 5, 81–102.
* Belsley D.A., Kuh, E. and Welsch, R.E. (1980) Regression Diagnostics. Identifying Influential Data and Sources of Collinearity. New York: Wiley.

Resources:
* A great more indepth blog post is Susan Li's [Building A Linear Regression with PySpark and MLlib](https://towardsdatascience.com/building-a-linear-regression-with-pyspark-and-mllib-d065c3ba246a)

## Load Data 
We can load our data using the Pandas syntax `.read_csv()` to read the CSV file and use the Pandas syntax `.head()` to review the top 10 rows of our Spark DataFrame.

In [0]:
df = sql("SELECT * FROM SAISNA_2020.boston_housing_delta")

In [0]:
# Convert from Spark DataFrame
pdf = df.toPandas()

# Display it
pdf.head(10)

,ID,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
3,5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
4,7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311,15.2,395.60,12.43,22.9
5,11,0.22489,12.5,7.87,0,0.524,6.377,94.3,6.3467,5,311,15.2,392.52,20.45,15.0
6,12,0.11747,12.5,7.87,0,0.524,6.009,82.9,6.2267,5,311,15.2,396.90,13.27,18.9
7,13,0.09378,12.5,7.87,0,0.524,5.889,39.0,5.4509,5,311,15.2,390.50,15.71,21.7
8,14,0.62976,0.0,8.14,0,0.538,5.949,61.8,4.7075,4,307,21.0,396.90,8.26,20.4
9,15,0.63796,0.0,8.14,0,0.538,6.096,84.5,4.4619,4,307,21.0,380.02,10.26,18.2


## Exploratory Data Analysis
Using `display` to explore the data

### Determine Possible Linear Correlation Between Multiple Independent Variables
Let's start off by creating a Scatterplot matrix of the different variables by using Databricks `display()`

In [0]:
display(pdf)

ID,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296,15.3,396.9,4.98,24.0
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.9,9.14,21.6
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.9,5.33,36.2
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311,15.2,395.6,12.43,22.9
11,0.22489,12.5,7.87,0,0.524,6.377,94.3,6.3467,5,311,15.2,392.52,20.45,15.0
12,0.11747,12.5,7.87,0,0.524,6.009,82.9,6.2267,5,311,15.2,396.9,13.27,18.9
13,0.09378,12.5,7.87,0,0.524,5.889,39.0,5.4509,5,311,15.2,390.5,15.71,21.7
14,0.62976,0.0,8.14,0,0.538,5.949,61.8,4.7075,4,307,21.0,396.9,8.26,20.4
15,0.63796,0.0,8.14,0,0.538,6.096,84.5,4.4619,4,307,21.0,380.02,10.26,18.2


#### `rm` and `medv`
In the above scatterplot, you can see a possible correlation between `rm` (average rooms per dwelling) and `medv` (median value of owner-occupied homes).  This can be better seen in the following scatterplot of just these two variables.

In [0]:
# View Scatterplot of data
display(pdf)

ID,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296,15.3,396.9,4.98,24.0
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.9,9.14,21.6
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.9,5.33,36.2
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311,15.2,395.6,12.43,22.9
11,0.22489,12.5,7.87,0,0.524,6.377,94.3,6.3467,5,311,15.2,392.52,20.45,15.0
12,0.11747,12.5,7.87,0,0.524,6.009,82.9,6.2267,5,311,15.2,396.9,13.27,18.9
13,0.09378,12.5,7.87,0,0.524,5.889,39.0,5.4509,5,311,15.2,390.5,15.71,21.7
14,0.62976,0.0,8.14,0,0.538,5.949,61.8,4.7075,4,307,21.0,396.9,8.26,20.4
15,0.63796,0.0,8.14,0,0.538,6.096,84.5,4.4619,4,307,21.0,380.02,10.26,18.2


#### `lstat` and `medv`
In the above scatterplot, you can see a possible negative linear correlation between `lstat` (lower status of population) and `medv` (median value of owner-occupied homes).  This can be better seen in the following scatterplot of just these two variables.

In [0]:
# View Scatterplot of data
display(pdf)

ID,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296,15.3,396.9,4.98,24.0
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.9,9.14,21.6
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.9,5.33,36.2
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311,15.2,395.6,12.43,22.9
11,0.22489,12.5,7.87,0,0.524,6.377,94.3,6.3467,5,311,15.2,392.52,20.45,15.0
12,0.11747,12.5,7.87,0,0.524,6.009,82.9,6.2267,5,311,15.2,396.9,13.27,18.9
13,0.09378,12.5,7.87,0,0.524,5.889,39.0,5.4509,5,311,15.2,390.5,15.71,21.7
14,0.62976,0.0,8.14,0,0.538,5.949,61.8,4.7075,4,307,21.0,396.9,8.26,20.4
15,0.63796,0.0,8.14,0,0.538,6.096,84.5,4.4619,4,307,21.0,380.02,10.26,18.2


#### Use Pandas `.corr` to Calculate Correlation Coefficents 
We can quickly calculate the correlation matrix of all attributes with `medv` using Pandas `.corr`.

In [0]:
# Calculate `corr`
pdf_corr = pdf.corr()

# Add Index 
pdf_corr['index1'] = pdf_corr.index

# Display values related to `medv`
display(pdf_corr.loc[:, ['index1', 'medv']])

index1,medv
ID,-0.19675465111492296
crim,-0.3421635325239667
zn,0.3584900525184189
indus,-0.46151316367831796
chas,0.2047684568842691
nox,-0.40108129302050644
rm,0.7294217505151407
age,-0.37178756492419596
dis,0.22021349164646534
rad,-0.34349753090870444


In [0]:
# New column names
column_names = ['ID', 'crime', 'zone', 'industry', 'bounds_river', 'nox', 'rooms', 'age', 'distance', 'radial_highway', 'tax', 'pupil_teacher', 'black_proportion', 'lower_status', 'median_value']

# Rename columns
pdf.columns = column_names

# Drop rows with median_value = NaN
pdf = pdf.dropna(subset=['median_value'])

### Choosing Features
Reviewing the correlation coefficient matrix and scatterplots, let's choose features that have slighty stronger positive or negative correlation to the `median_value` where `abs(correlation coefficients) >= 0.4`.

In [0]:
# All columns but median_value
featureColumns = ['crime', 'zone', 'industry', 'bounds_river', 'nox', 'rooms', 'age', 'distance', 'radial_highway', 'tax', 'pupil_teacher', 'black_proportion', 'lower_status']

## Build our Linear Regression Model

Let's build our Linear Regression model using Scikit-Learn.

In [0]:
# Split training and test datasets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(pdf[featureColumns].values, pdf['median_value'].values, test_size=0.20, random_state=567248)

## Evaluate Model Performance
Calculate [RMSE](https://en.wikipedia.org/wiki/Root-mean-square_deviation) and [r<sup>2</sup>](https://en.wikipedia.org/wiki/Coefficient_of_determination)

In [0]:
# Split the data into training and test sets.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(pdf[featureColumns].values, pdf['median_value'].values, test_size=0.20, random_state=567248)  

In [0]:
def train_boston_housing(X_train, X_test, y_train, y_test, type, alpha):
  from sklearn.linear_model import Ridge, Lasso
  from sklearn.metrics import mean_squared_error, r2_score
  from yellowbrick.regressor import PredictionError, ResidualsPlot
  import math
  import matplotlib.pyplot as plt
  
  # Start an MLflow run; the "with" keyword ensures we'll close the run even if this cell crashes
  with mlflow.start_run() as run:
    # Create linear regression object
    if (type == "ridge"):
      lr = Ridge(alpha=alpha)
    else:
      lr = Lasso(alpha=alpha)

    # Train the model using the training sets
    lr.fit(X_train, y_train)

    # Make predictions using the testing set
    y_pred = lr.predict(X_test)

    # Calculate RMSE and r2
    rmse = math.sqrt(mean_squared_error(y_test, y_pred))
    r2 =  r2_score(y_test, y_pred)
    
    # Calculate RMSE
    print("RMSE: %.3f" % rmse)
    print("R^2: %.3f" % r2)    
  
    # Log mlflow attributes for mlflow UI
    mlflow.log_param("type", type)
    mlflow.log_param("alpha", alpha)    
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.sklearn.log_model(lr, "model")
    
    # Prediction Error plot
    plt.gcf().clear()
    visualizer_pe = PredictionError(lr, size=(1000,800))
    visualizer_pe.fit(X_train, y_train)  # Fit the training data to the visualizer
    visualizer_pe.score(X_test, y_test)  # Evaluate the model on the test data
    visualizer_pe.poof("prediction-error.png")                 # Finalize and render the figure  
    
    # Log artifacts (output files)
    mlflow.log_artifact("prediction-error.png")

    # Residuals Plot
    plt.gcf().clear()
    visualizer_re = ResidualsPlot(lr, size=(1000, 800))
    visualizer_re.fit(X_train, y_train)  # Fit the training data to the model
    visualizer_re.score(X_test, y_test)  # Evaluate the model on the test data
    visualizer_re.show("residuals.png")                 # Draw/show/poof the data    

    # Log artifacts (output files)
    mlflow.log_artifact("residuals.png")

    # return our RunUUID so we can use it when we try out some other APIs later in this notebook.
    return run.info

In [0]:
# Cross Validation List
cv_list = [1.0, 0.5, 0.1, 0.01, 0.005, 0.001]

In [0]:
# Using Ridge
for i in cv_list:
  train_boston_housing(X_train, X_test, y_train, y_test, "ridge", i)

In [0]:
# Using Ridge
for i in cv_list:
  train_boston_housing(X_train, X_test, y_train, y_test, "lasso", i)  

## Load Production MLflow model back as a Scikit-learn model from MLflow Registry 
Go to MLflow Model Registry:
* Go to MLflow and register model
* Transition model to production

In [0]:
import mlflow.pyfunc
model_name = "Boston Housing Ridge"
model_production_uri = "models:/{model_name}/production".format(model_name=model_name)
print("Loading registered model version from URI: '{model_uri}'".format(model_uri=model_production_uri))
model_production = mlflow.pyfunc.load_model(model_production_uri)

# Loading the model from Model Registry
print (model_production_uri)
model_production.coef_

#Get a prediction for a row of the dataset
model_production.predict(X_test[0:1])

Loading registered model version from URI: 'models:/Boston Housing Ridge/production'
models:/Boston Housing Ridge/production
Out[89]: array([20.87050614])